# 3 - Análise exploratória e Consultas SQL

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import duckdb
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

### Carregando dados

In [ ]:
df = pd.read_parquet('arrests_limpo.parquet', engine='fastparquet')

df = df.replace('(null)', np.nan)

print("--- ESTADO INICIAL (ANTES DO TIDYING) ---")
df.info()
df.head(5)

## Transformações estruturais

In [ ]:
# Não precisa.

## Normalização

In [ ]:
print("Iniciando Padronização (Pré-requisitos)...")

print("Convertendo 'arrest_date' para datetime...")
df['arrest_date'] = pd.to_datetime(df['arrest_date'], errors='coerce')

print("Convertendo coordenadas para numérico...")
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

print("Padronização inicial concluída.")

print("Iniciando Normalização...")

print("Separando 'arrest_date' em componentes...")
df['arrest_year'] = df['arrest_date'].dt.year
df['arrest_month'] = df['arrest_date'].dt.month
df['arrest_day'] = df['arrest_date'].dt.day
df['arrest_day_of_week'] = df['arrest_date'].dt.day_name()
df['arrest_hour'] = df['arrest_date'].dt.hour

print("Removendo coluna aninhada 'geocoded_column'...")
if 'geocoded_column' in df.columns:
    df = df.drop(columns=['geocoded_column'])

print("Normalização concluída.")

## Padronização de tipos

In [ ]:
print("Iniciando Padronização (Otimização)...")

print("Convertendo IDs (arrest_key, arrest_precinct) para string...")
df['arrest_key'] = df['arrest_key'].astype(str)
df['arrest_precinct'] = df['arrest_precinct'].astype(str)

print("Convertendo colunas de alta repetição para 'category'...")
colunas_categoricas = [
    'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc', 'law_code',
    'law_cat_cd', 'arrest_boro', 'jurisdiction_code',
    'age_group', 'perp_sex', 'perp_race', 'arrest_day_of_week'
]

for col in colunas_categoricas:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("Padronização (Parte 2) concluída.")

print("--- ESTADO FINAL (DEPOIS DO TIDYING) ---")
df.info()

## Exportação de dados preparados em Parquet

In [ ]:
print("Exportando dados preparados...")
df.to_parquet('arrests_tidy_prepared.parquet', engine='fastparquet')
print("Arquivo 'arrests_tidy_prepared.parquet' salvo com sucesso!")

print("\nAmostra dos dados finais:")
df.head()

## Consultas SQL Analíticas

### --- 1. TENDÊNCIA: Média Móvel de 7 dias ---

Pergunta: "Qual a média móvel de prisões? As prisões estão aumentando ou diminuindo este ano?"

In [ ]:
con = duckdb.connect(database=':memory:')

sql_trend = """
WITH daily_counts AS (
    -- 1. CTE: Agrega o total de prisões por dia
    SELECT
        CAST(arrest_date AS DATE) as date,
        COUNT(arrest_key) as total_arrests
    FROM df
    GROUP BY 1
)
-- 2. Window Function: Calcula a média dos 7 dias anteriores
SELECT
    date,
    total_arrests,
    AVG(total_arrests) OVER (
        ORDER BY date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as moving_avg_7_day
FROM daily_counts
ORDER BY date DESC
"""

# Rode e veja o resultado
trend_df = con.execute(sql_trend).df()
trend_df

### --- 2. COMPARAÇÃO: Top 3 Crimes por Distrito ---

Pergunta: "Quais são os 3 principais tipos de crime (descrição da ofensa) em cada Distrito (Borough)?"

In [ ]:
sql_top_crimes = """
WITH crime_counts AS (
    -- 1. CTE: Conta cada crime em cada distrito
    SELECT
        arrest_boro,
        ofns_desc, -- Usando a descrição da ofensa
        COUNT(*) as crime_count
    FROM df
    WHERE arrest_boro IS NOT NULL AND ofns_desc IS NOT NULL
    GROUP BY 1, 2
),
ranked_crimes AS (
    -- 2. Window Function: Cria um ranking (1, 2, 3...)
    -- para cada crime DENTRO de cada distrito
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY arrest_boro -- Reinicia o ranking para cada distrito
            ORDER BY crime_count DESC -- O mais comum fica em 1º
        ) as rank
    FROM crime_counts
)
-- 3. Filtro Final: Pega só o Top 3
SELECT
    arrest_boro,
    ofns_desc,
    crime_count,
    rank
FROM ranked_crimes
WHERE rank <= 3
ORDER BY arrest_boro, rank;
"""

top_crimes_df = con.execute(sql_top_crimes).df()
top_crimes_df

### --- 3. CONCENTRAÇÃO: Delegacias "Hotspot" (Pareto) ---

Pergunta: "Quantas delegacias (precincts) são responsáveis por 50% de todas as prisões? Onde o policiamento está concentrado?"

In [ ]:
sql_concentration = """
WITH precinct_counts AS (
    -- 1. CTE: Conta o total por delegacia
    SELECT
        arrest_precinct,
        COUNT(*) as total_arrests
    FROM df
    GROUP BY 1
),
cumulative_percent AS (
    -- 2. CTE + Window Function: Calcula o % acumulado
    SELECT
        arrest_precinct,
        total_arrests,
        -- Soma o total de prisões em ordem decrescente
        SUM(total_arrests) OVER (ORDER BY total_arrests DESC) as cumulative_arrests,
        -- Divide pelo total geral para ter o %
        100.0 * SUM(total_arrests) OVER (ORDER BY total_arrests DESC) / SUM(total_arrests) OVER () as cumulative_pct
    FROM precinct_counts
)
-- 3. Filtro Final: Mostra só as delegacias que somam os primeiros 50%
SELECT
    arrest_precinct,
    total_arrests,
    cumulative_pct
FROM cumulative_percent
WHERE cumulative_pct <= 50  -- Mude para 80 se quiser o "Princípio de Pareto"
ORDER BY total_arrests DESC;
"""

concentration_df = con.execute(sql_concentration).df()
concentration_df

### --- 4. PADRÃO: Análise de Hora vs. Dia da Semana ---

Pergunta: "Prisões por FELONY ASSAULT (Agressão Grave) acontecem mais de dia ou de madrugada? E nos fins de semana?"

In [ ]:
sql_pattern = """
-- Criamos "faixas de horário" para simplificar
WITH time_slots AS (
    SELECT
        *,
        CASE
            WHEN arrest_hour BETWEEN 6 AND 11 THEN '1. Manhã (06-11)'
            WHEN arrest_hour BETWEEN 12 AND 17 THEN '2. Tarde (12-17)'
            WHEN arrest_hour BETWEEN 18 AND 23 THEN '3. Noite (18-23)'
            ELSE '4. Madrugada (00-05)'
        END as time_of_day
    FROM df
    WHERE ofns_desc = 'FELONY ASSAULT' -- Filtre pelo crime que quer analisar
)
SELECT
    time_of_day,
    arrest_day_of_week,
    COUNT(*) as total_assault_arrests
FROM time_slots
WHERE arrest_day_of_week IS NOT NULL
GROUP BY 1, 2
ORDER BY 1, 3 DESC; -- Ordena pela faixa de horário, depois pelo total
"""

pattern_df = con.execute(sql_pattern).df()
pattern_df

### --- 5. DEPENDÊNCIA: Faixa Etária vs. Gravidade do Crime ---

Pergunta: "Existe uma relação visível entre a faixa etária (age_group) e a gravidade do crime (law_cat_cd)?"

In [ ]:
sql_crosstab = """
SELECT
    age_group,
    -- Contagem para cada categoria de lei (usando PIVOT manual)
    COUNT(CASE WHEN law_cat_cd = 'F' THEN 1 END) as felony,
    COUNT(CASE WHEN law_cat_cd = 'M' THEN 1 END) as misdemeanor,
    COUNT(CASE WHEN law_cat_cd = 'V' THEN 1 END) as violation,
    COUNT(*) as total,
    
    -- % de prisões daquele grupo que foram Felonies (o insight real)
    ROUND(100.0 * COUNT(CASE WHEN law_cat_cd = 'F' THEN 1 END) / COUNT(*), 2) as pct_felony
FROM df
WHERE age_group IS NOT NULL
GROUP BY 1
ORDER BY pct_felony DESC; -- Ordena por quem tem a maior % de crimes graves
"""

crosstab_df = con.execute(sql_crosstab).df()
crosstab_df

## Análise Exploratória e Teste de Hipóteses

## 4.1 Análise Univariada (Conhecendo as Peças)

Como os dados se distribuem em uma única variável?

In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6) # Gráficos maiores

# (a) Variável Categórica: 'arrest_boro' (Distrito)
# Vamos ver a distribuição percentual de prisões
print("--- Distribuição de Prisões por Distrito ---")
print(df['arrest_boro'].value_counts(normalize=True).mul(100).round(2).to_frame('% Total'))

# Visualização
sns.countplot(data=df, x='arrest_boro', order=df['arrest_boro'].value_counts().index)
plt.title('Distribuição de Prisões por Distrito (Borough)')
plt.ylabel('Total de Prisões')
plt.xlabel('Distrito')
plt.show()

# %%
# (b) Variável Numérica (Tidy): 'arrest_hour' (Hora da Prisão)
# Isso mostra padrões de "hora do rush" de prisões
print("\n--- Distribuição de Prisões por Hora ---")

# Visualização
sns.histplot(data=df, x='arrest_hour', bins=24, kde=True)
plt.title('Distribuição de Prisões por Hora do Dia')
plt.xlabel('Hora (0-23)')
plt.ylabel('Contagem')
plt.xticks(range(0, 24))
plt.show()

## 4.2 Análise Bivariada e Teste de Hipóteses

### H1: Pessoas mais jovens cometem mais crimes que pessoas mais velhas.

In [ ]:
h1_sql = """
SELECT
    age_group,
    COUNT(arrest_key) as total_arrests
FROM df
WHERE age_group IS NOT NULL -- Ignora os nulos que limpamos
GROUP BY 1
ORDER BY 2 DESC
"""
h1_df = con.execute(h1_sql).df()
print(h1_df)

# Visualização
sns.barplot(data=h1_df, x='age_group', y='total_arrests', order=h1_df.sort_values('total_arrests', ascending=False)['age_group'])
plt.title('H1: Total de Prisões por Faixa Etária')
plt.ylabel('Total de Prisões')
plt.xlabel('Faixa Etária')
plt.show()

### H2: Asiáticos cometem menos crimes que outras etnias.

In [ ]:
h2_sql = """
SELECT
    perp_race,
    COUNT(arrest_key) as total_arrests
FROM df
WHERE perp_race IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC -- Ordena do maior para o menor
"""
h2_df = con.execute(h2_sql).df()
print(h2_df)

# Visualização (gráfico de barras horizontais é melhor aqui)
sns.barplot(data=h2_df, y='perp_race', x='total_arrests', order=h2_df.sort_values('total_arrests', ascending=False)['perp_race'])
plt.title('H2: Total de Prisões por Etnia')
plt.xlabel('Total de Prisões')
plt.ylabel('Etnia (Perp. Race)')
plt.show()

### H3: Acontece mais crime no verão do que no inverno.

In [ ]:
h3_sql = """
-- Criamos 'temporada' (estação) para facilitar
WITH seasonal_arrests AS (
    SELECT
        arrest_key,
        arrest_month,
        CASE
            WHEN arrest_month IN (12, 1, 2) THEN '1. Inverno (Dez-Fev)'
            WHEN arrest_month IN (3, 4, 5) THEN '2. Primavera (Mar-Mai)'
            WHEN arrest_month IN (6, 7, 8) THEN '3. Verão (Jun-Ago)'
            WHEN arrest_month IN (9, 10, 11) THEN '4. Outono (Set-Nov)'
        END as season
    FROM df
    WHERE arrest_month IS NOT NULL
)
SELECT
    season,
    COUNT(arrest_key) as total_arrests
FROM seasonal_arrests
GROUP BY 1
ORDER BY 1 -- Ordena pelas estações
"""
h3_df = con.execute(h3_sql).df()
print(h3_df)

# Visualização (Gráfico de linha é ótimo para sazonalidade)
sns.lineplot(data=h3_df, x='season', y='total_arrests', marker='o')
plt.title('H3: Sazonalidade das Prisões (Hemisfério Norte)')
plt.ylabel('Total de Prisões')
plt.xlabel('Estação do Ano')
plt.show()

## Análise Multivariada

In [ ]:
crosstab_df = pd.crosstab(
    df['arrest_boro'],      # Linhas
    df['law_cat_cd'],       # Colunas
    normalize='index'       # Normaliza pela linha (mostra %)
)
crosstab_df = crosstab_df * 100 # Transforma em porcentagem (0-100)

print(crosstab_df.round(1))

# Visualização: Heatmap
plt.figure(figsize=(10, 5))
sns.heatmap(crosstab_df, annot=True, fmt='.1f', cmap='viridis')
plt.title('Heatmap: % de Gravidade do Crime por Distrito')
plt.ylabel('Distrito (Borough)')
plt.xlabel('Gravidade da Lei (F=Grave, M=Médio, V=Leve)')
plt.show()